<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/VGG19_with_avgpooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -q
!wandb login

     |████████████████████████████████| 1.9 MB 29.1 MB/s 
     |████████████████████████████████| 168 kB 74.0 MB/s 
     |████████████████████████████████| 182 kB 64.9 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 64.2 MB/s 
     |████████████████████████████████| 166 kB 71.0 MB/s 
     |████████████████████████████████| 166 kB 77.9 MB/s 
     |████████████████████████████████| 162 kB 71.4 MB/s 
     |████████████████████████████████| 162 kB 74.7 MB/s 
     |████████████████████████████████| 158 kB 74.4 MB/s 
     |████████████████████████████████| 157 kB 73.5 MB/s 
     |████████████████████████████████| 157 kB 72.5 MB/s 
     |████████████████████████████████| 157 kB 73.2 MB/s 
     |████████████████████████████████| 157 kB 77.6 MB/s 
     |████████████████████████████████| 157 kB 73.6 MB/s 
     |████████████████████████████████| 157 kB 64.2 MB/s 
     |████████████████████████████████| 157 kB 59.5 MB/s 
     |██████████

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(224,224,3))

80134624/80134624 [==============================] - 0s 0us/step


In [6]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 100 files belonging to 5 classes.


In [7]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
#class_names == test_class_names
num_classes = len(class_names)
print(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [9]:
model_vgg19 = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  vgg19,
#   layers.Flatten(),
#   layers.Dense(32, activation='relu'),
  layers.GlobalAveragePooling2D(),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
vgg19.trainable=False
model_vgg19.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 20,026,949
Trainable params: 2,565
Non-trainable params: 20,024,384
______________________________________

In [10]:
model_vgg19.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [11]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "vgg19_with_avgpooling",
)

wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [12]:
%%time
epochs=40
history = model_vgg19.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.8332 - accuracy: 0.2074

wandb: Adding directory to artifact (/content/wandb/run-20221128_070904-1ocmc5k0/files/model-best)... Done. 0.7s


22/22 [==============================] - 118s 3s/step - loss: 1.8332 - accuracy: 0.2074 - val_loss: 1.3121 - val_accuracy: 0.5600
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.2476 - accuracy: 0.5582

wandb: Adding directory to artifact (/content/wandb/run-20221128_070904-1ocmc5k0/files/model-best)... Done. 0.7s


...

22/22 [==============================] - 13s 592ms/step - loss: 0.2998 - accuracy: 0.9261 - val_loss: 0.2993 - val_accuracy: 0.9200
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.2914 - accuracy: 0.9318

wandb: Adding directory to artifact (/content/wandb/run-20221128_070904-1ocmc5k0/files/model-best)... Done. 0.7s


22/22 [==============================] - 13s 585ms/step - loss: 0.2914 - accuracy: 0.9318 - val_loss: 0.2991 - val_accuracy: 0.9086
Epoch 40/40
22/22 [==============================] - ETA: 0s - loss: 0.2925 - accuracy: 0.9233

wandb: Adding directory to artifact (/content/wandb/run-20221128_070904-1ocmc5k0/files/model-best)... Done. 0.7s


22/22 [==============================] - 13s 578ms/step - loss: 0.2925 - accuracy: 0.9233 - val_loss: 0.2928 - val_accuracy: 0.9029
CPU times: user 3min 32s, sys: 42.5 s, total: 4min 15s
Wall time: 13min 38s


In [13]:
wandb.finish()

accuracy,▁▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇██▇█▇███████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▃▃▃▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████████
val_loss,█▇▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.9233
best_epoch,39
best_val_loss,0.29282
epoch,39
loss,0.29249
val_accuracy,0.90286


In [16]:
wandb.alert(
    title="Run Finished", 
    text = "Run Finished"
)

In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
